In [46]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [67]:
df = pd.read_csv('C:/MSDA/data245/Coding/data/train.csv')
print(df.columns)
df.drop(columns=['PassengerId','Name','Ticket','Fare','Cabin','Age'], inplace = True) # drop Cabin ?
df['Pclass'] = df['Pclass'].replace([1,2,3], ['1st', '2nd', '3rd'])
df['Survived'] = df['Survived'].replace([1,0], ['T', 'F'])
df['SibSp'] = df['SibSp'].replace([0, 1, 2, 3, 4, 5, 6, 7, 8], ['zero','one', 'two','three','four','five','six','seven','eight'])
df['Parch'] = df['Parch'].replace([0, 1, 2, 3, 4, 5, 6, 7, 8], ['zero','one', 'two','three','four','five','six','seven','eight'])

'''X = df.drop(columns=['Survived'])
Y = df.Survived'''
df

Index(['PassengerId', 'Survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Ticket', 'Fare', 'Cabin', 'Embarked'],
      dtype='object')


,Survived,Pclass,Sex,SibSp,Parch,Embarked
0,F,3rd,male,one,zero,S
1,T,1st,female,one,zero,C
2,T,3rd,female,zero,zero,S
3,T,1st,female,one,zero,S
4,F,3rd,male,zero,zero,S
...,...,...,...,...,...,...
886,F,2nd,male,zero,zero,S
887,T,1st,female,zero,zero,S
888,F,3rd,female,one,two,S
889,T,1st,male,zero,zero,C


In [164]:
df['Embarked'] = df['Embarked'].replace(np.NaN, 'nan')

In [165]:
df.isnull().sum()

Survived    0
Pclass      0
Sex         0
SibSp       0
Parch       0
Embarked    0
dtype: int64

In [166]:
df[df.Embarked.isnull()]

,Survived,Pclass,Sex,SibSp,Parch,Embarked


In [63]:
df.describe()

,Survived,Pclass,Sex,SibSp,Parch,Embarked
count,891,891,891,891,891,889
unique,2,3,2,7,7,3
top,F,3rd,male,zero,zero,S
freq,549,491,577,608,678,644


In [144]:
df[df.Embarked == 'C']

,Survived,Pclass,Sex,SibSp,Parch,Embarked
1,T,1st,female,one,zero,C
9,T,2nd,female,one,zero,C
19,T,3rd,female,zero,zero,C
26,F,3rd,male,zero,zero,C
30,F,1st,male,zero,zero,C
...,...,...,...,...,...,...
866,T,2nd,female,one,zero,C
874,T,2nd,female,one,zero,C
875,T,3rd,female,zero,zero,C
879,T,1st,female,zero,one,C


In [105]:
def entropy(l):
    return(-sum([0 if i == 0 else i*np.log2(i) for i in l])/len(l))


In [189]:
def DT(df,attribute):
    entropy_node = 0
    t_values = df.Survived.unique()
    fraction_t = []
    for value in t_values:
        fraction_t.append(df.Survived.value_counts()[value]/len(df.Survived))
    entropy_node = entropy(fraction_t)
    variables = np.unique(list(df[attribute]))
    w = []
    ent_p = []
    for variable in variables:
        fraction = []
        for target_variable in t_values:
            num = len(df[attribute][df[attribute]==variable][df.Survived ==target_variable]) #numerator
            den = len(df[attribute][df[attribute]==variable])  #denominator
            print(target_variable, variable, num, den, num/den)
            fraction.append(np.array(num/den))
        ent_p.append(entropy(fraction))
        w.append(den/len(df)) # weight
    #print('entropy',ent_p, 'w', w, 'fraction', fraction)
    ent_rem = 0
    for i, j in zip(w, ent_p):
        ent_rem += i*j
    ig = entropy_node - ent_rem
    return ig

    #print(f'For {attribute}:\nTotal: {entropy_node}\npartition entropy:')
    for i, j in zip(variables, ent_p):
        print(i,j,  end = ' ')
    print(f'\nRem.: {ent_rem: .4}\nIG: {ig: 0.4}\n\n')


In [190]:
ig_d = {k:DT(df,k) for k in df.keys()[1:]}
#[DT(df, k) for k in df.keys()[1:]]
ig_d

F 1st 80 216 0.37037037037037035
T 1st 136 216 0.6296296296296297
F 2nd 97 184 0.5271739130434783
T 2nd 87 184 0.47282608695652173
F 3rd 372 491 0.7576374745417516
T 3rd 119 491 0.24236252545824846
F female 81 314 0.25796178343949044
T female 233 314 0.7420382165605095
F male 468 577 0.8110918544194108
T male 109 577 0.18890814558058924
F eight 7 7 1.0
T eight 0 7 0.0
F five 5 5 1.0
T five 0 5 0.0
F four 15 18 0.8333333333333334
T four 3 18 0.16666666666666666
F one 97 209 0.46411483253588515
T one 112 209 0.5358851674641149
F three 12 16 0.75
T three 4 16 0.25
F two 15 28 0.5357142857142857
T two 13 28 0.4642857142857143
F zero 398 608 0.6546052631578947
T zero 210 608 0.34539473684210525
F five 4 5 0.8
T five 1 5 0.2
F four 4 4 1.0
T four 0 4 0.0
F one 53 118 0.4491525423728814
T one 65 118 0.5508474576271186
F six 1 1 1.0
T six 0 1 0.0
F three 2 5 0.4
T three 3 5 0.6
F two 40 80 0.5
T two 40 80 0.5
F zero 445 678 0.6563421828908554
T zero 233 678 0.34365781710914456
F C 75 168 0.446

{'Pclass': 0.0419155226480058,
 'Sex': 0.1088300533303071,
 'SibSp': 0.016733160669354985,
 'Parch': 0.011805273816735296,
 'Embarked': 0.012023545353980258}

In [193]:
dfmale = df[df.Sex == 'male'].drop(columns = ['Sex'])
dffemale = df[df.Sex == 'female'].drop(columns = ['Sex'])
root_node = 'Sex'

In [192]:
ig_sex1 = {k:DT(df,k) for k in dfmale.keys()[1:]}
ig_sex2 = {k:DT(df,k) for k in dffemale.keys()[1:]}
print(ig_sex1)
print(ig_sex2)

F 1st 80 216 0.37037037037037035
T 1st 136 216 0.6296296296296297
F 2nd 97 184 0.5271739130434783
T 2nd 87 184 0.47282608695652173
F 3rd 372 491 0.7576374745417516
T 3rd 119 491 0.24236252545824846
F eight 7 7 1.0
T eight 0 7 0.0
F five 5 5 1.0
T five 0 5 0.0
F four 15 18 0.8333333333333334
T four 3 18 0.16666666666666666
F one 97 209 0.46411483253588515
T one 112 209 0.5358851674641149
F three 12 16 0.75
T three 4 16 0.25
F two 15 28 0.5357142857142857
T two 13 28 0.4642857142857143
F zero 398 608 0.6546052631578947
T zero 210 608 0.34539473684210525
F five 4 5 0.8
T five 1 5 0.2
F four 4 4 1.0
T four 0 4 0.0
F one 53 118 0.4491525423728814
T one 65 118 0.5508474576271186
F six 1 1 1.0
T six 0 1 0.0
F three 2 5 0.4
T three 3 5 0.6
F two 40 80 0.5
T two 40 80 0.5
F zero 445 678 0.6563421828908554
T zero 233 678 0.34365781710914456
F C 75 168 0.44642857142857145
T C 93 168 0.5535714285714286
F Q 47 77 0.6103896103896104
T Q 30 77 0.38961038961038963
F S 427 644 0.6630434782608695
T S 21

In [186]:
df[(df.Sex == 'male')&(df.Pclass == '1st')].Survived.value_counts()

F    77
T    45
Name: Survived, dtype: int64